In [43]:
!pip install -q sentence-transformers langchain chromadb pypdf faiss-cpu \
langchain_community scikit-learn numpy mistralai langchain-mistralai


In [44]:
!pip install pypdf


In [35]:
import os
import json
import time
from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings


In [4]:
import json
json_keys={}
with open("./keys.json","r") as f:
  json_keys=json.load(f)

In [5]:

os.environ["GOOGLE_API_KEY"] = json_keys["googleapi"]
os.environ["GOOGLE_CSE_ID"] = json_keys["cseapi"]
os.environ["MISTRAL_API_KEY"]=json_keys["mistralapi"]
os.environ["OPENWEATHERMAP_API_KEY"] = json_keys["openweatherapi"]
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./credentials.json"

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./ib-acio-2025-notification.pdf")
docs = loader.load()
len(docs)


21

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(docs)
len(chunks)


129

In [11]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_db = FAISS.from_documents(chunks, embedding=embeddings)
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})


/tmp/ipython-input-2603739455.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)


/tmp/ipython-input-3186262939.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [13]:
template = """
You are a Memory-Augmented RAG assistant.

Conversation History:
{chat_history}

Relevant Retrieved Context:
{context}

User Query:
{question}

Provide a context-aware answer. ALWAYS use history if relevant.
"""

prompt = PromptTemplate(
    input_variables=["chat_history", "context", "question"],
    template=template
)


In [14]:
llm = ChatMistralAI(model="mistral-large-latest", temperature=0)

rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt}
)


In [16]:
answers = {
    "Questions":[],
    "Answers":[],
    "Response Time":[]
}


In [17]:

while True:
  q = input("You: ")
  if q.lower() in ["exit", "quit"]:
      break
  start_time = time.time()

  resp = rag_chain.invoke({"question": q})

  end_time = time.time()
  duration = end_time - start_time

  answers["Questions"].append(q)
  answers["Answers"].append(resp["answer"])
  answers["Response Time"].append(f"{duration:.2f} seconds")

  print("Bot:", resp["answer"])


You: what is name of the Post ?
Bot: Based on the **conversation history** and the retrieved context, the **recruitment notification does not explicitly mention the name of the post** in the provided snippets.

However, the context strongly suggests it is related to a **government recruitment process under the Ministry of Home Affairs (MHA)**, likely for positions such as:
- **Assistant Central Intelligence Officer (ACIO)**
- **Sub-Inspector (SI) in Central Armed Police Forces (CAPF)**
- **Other Group B/C posts** (e.g., Stenographer, Ministerial Staff, etc.)

### Key Observations:
1. The notification refers to **reservation claims (SC/ST/OBC/PwD/Ex-servicemen)**, which are common in MHA recruitments.
2. The **self-declaration form** (seen in the context) is typical for posts requiring security clearance (e.g., intelligence or police roles).
3. The **Application Sequence Number** and strict communication protocols align with MHA’s recruitment portals (e.g., [mha.gov.in](https://www.mha.

In [31]:
memory_usage = memory.buffer_as_messages
for m in range(0,len(memory_usage)-1,2):
  print("User Query ===> ", memory_usage[m].content)
  print()
  print("Bot Response ===> ", memory_usage[m+1].content)
  print()
  print("="*190)

User Query ===>  what is these notification about ?

Bot Response ===>  Based on the retrieved context, this notification appears to be related to a **recruitment process** (likely for a government exam or position under the **Ministry of Home Affairs (MHA)**). Here are the key points it covers:

1. **Reservation/Relaxation Claims**:
   - Candidates claiming reservation or relaxation (e.g., for SC/ST/OBC/PwD categories) must ensure they are **legally entitled** to it.
   - They must possess **valid certificates** in the **Government of India’s prescribed format** (referenced as Appendix-1 to 5) and submit them when required.

2. **Communication Requirements**:
   - Candidates must provide **one active mobile number** for SMS alerts about the exam.
   - The recruiting office **will not be responsible** if emails bounce or if the mobile number malfunctions/changes.

3. **Updates and Responsibilities**:
   - Candidates must **regularly check their email (including spam folders)** and the 

In [33]:
query = input("Enter the Query: ")
resp = rag_chain.invoke({"question": query})
print(resp["answer"])

Enter the Query: i forgot about previous query that i ask, can you repeat my query ?
### **🔍 Your Previous Questions (Summary)**
1. **First Question**:
   *"What is this notification about?"*
   → **Answer**: Recruitment for **MHA posts (likely ACIO-II/Exe)** with rules on **reservation, certificates, and communication**.

2. **Second Question**:
   *"What is the name of the post?"*
   → **Answer**: **ACIO-II/Exe (Assistant Central Intelligence Officer Grade-II/Executive)** under **Intelligence Bureau (IB), MHA**.

3. **Third Question**:
   *"What is the eligibility criteria for this post?"*
   → **Answer**: **Age 18–27**, **bachelor’s degree**, **Indian citizen**, **valid ID proof**, **no PwD**, **clean record**, etc.

4. **Fourth Question (Current)**:
   *"I’m 17 with a bachelor’s degree, no PAN/Voter ID/birth certificate. Am I eligible?"*
   → **Answer**: **No** (due to **age <18** and missing IDs). **Wait until 18**, then get **Aadhar/PAN** and apply.

---

### **🚨 Why You’re Not E

In [34]:
memory_usage = memory.buffer_as_messages
for m in range(0,len(memory_usage)-1,2):
  print("User Query ===> ", memory_usage[m].content)
  print()
  print("Bot Response ===> ", memory_usage[m+1].content)
  print()
  print("="*190)

User Query ===>  what is these notification about ?

Bot Response ===>  Based on the retrieved context, this notification appears to be related to a **recruitment process** (likely for a government exam or position under the **Ministry of Home Affairs (MHA)**). Here are the key points it covers:

1. **Reservation/Relaxation Claims**:
   - Candidates claiming reservation or relaxation (e.g., for SC/ST/OBC/PwD categories) must ensure they are **legally entitled** to it.
   - They must possess **valid certificates** in the **Government of India’s prescribed format** (referenced as Appendix-1 to 5) and submit them when required.

2. **Communication Requirements**:
   - Candidates must provide **one active mobile number** for SMS alerts about the exam.
   - The recruiting office **will not be responsible** if emails bounce or if the mobile number malfunctions/changes.

3. **Updates and Responsibilities**:
   - Candidates must **regularly check their email (including spam folders)** and the 